In [1]:
#importing spotify libraries
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
cid ="xx" 
secret = "xx"

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [4]:
# timeit library to measure the time needed to run this code
import timeit
start = timeit.default_timer()

# create empty lists where the results are going to be stored
artist_name = []
track_name = []
popularity = []
track_id = []

In [5]:
for i in range(0,10000,50):
    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])
      

stop = timeit.default_timer()
print ('Time to run this code (in seconds):', stop - start)

Time to run this code (in seconds): 162.18516139500025


In [6]:
print('number of elements in the track_id list:', len(track_id))

number of elements in the track_id list: 10000


In [7]:
import pandas as pd

df_tracks = pd.DataFrame({'artist_name':artist_name,'track_name':track_name,'track_id':track_id,'popularity':popularity})
print(df_tracks.shape)
df_tracks.head()

(10000, 4)


,artist_name,track_name,track_id,popularity
0,Shawn Mendes,Señorita,0TK2YIli7K1leLovkQiNik,100
1,Drake,Money In The Grave (Drake ft. Rick Ross),5ry2OE6R2zPQFDO85XkgRb,92
2,Lizzo,Truth Hurts,5qmq61DAAOUaW8AUo8xKhh,90
3,Lil Nas X,Panini,6fTt0CH2t0mdeB2N9XFG5r,91
4,Billie Eilish,bad guy,2Fxmhks0bxGSBdJ92vM42m,96


In [8]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
artist_name    10000 non-null object
track_name     10000 non-null object
track_id       10000 non-null object
popularity     10000 non-null int64
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [9]:
# check for duplicates
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'])].count()

artist_name    1295
track_name     1295
track_id       1295
popularity     1295
dtype: int64

In [10]:
df_tracks.drop_duplicates(subset=['artist_name','track_name'], inplace=True)

In [11]:
# check for duplicates again
df_tracks[df_tracks.duplicated(subset=['artist_name','track_name'])].count()

artist_name    0
track_name     0
track_id       0
popularity     0
dtype: int64

In [12]:
df_tracks.shape

(8705, 4)

In [13]:
# again measuring the time
start = timeit.default_timer()

# empty list, batchsize and the counter for None results
rows = []
batchsize = 100
None_counter = 0

for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)
            
print('Number of tracks where no audio features were available:',None_counter)

stop = timeit.default_timer()
print ('Time to run this code (in seconds):',stop - start)

Number of tracks where no audio features were available: 204
Time to run this code (in seconds): 26.998793757000385


In [14]:
print('number of elements in the track_id list:', len(rows))

number of elements in the track_id list: 8501


In [15]:
df_audio_features = pd.DataFrame.from_dict(rows,orient='columns')
print("Shape of the dataset:", df_audio_features.shape)
df_audio_features.head()

Shape of the dataset: (8501, 18)


,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
0,0.037,https://api.spotify.com/v1/audio-analysis/0TK2...,0.759,190960,0.540,0TK2YIli7K1leLovkQiNik,0.00,9,0.0945,-6.039,0,0.0287,116.947,4,https://api.spotify.com/v1/tracks/0TK2YIli7K1l...,audio_features,spotify:track:0TK2YIli7K1leLovkQiNik,0.750
1,0.101,https://api.spotify.com/v1/audio-analysis/5ry2...,0.831,205427,0.502,5ry2OE6R2zPQFDO85XkgRb,0.00,10,0.1220,-4.045,0,0.0460,100.541,4,https://api.spotify.com/v1/tracks/5ry2OE6R2zPQ...,audio_features,spotify:track:5ry2OE6R2zPQFDO85XkgRb,0.101
2,0.110,https://api.spotify.com/v1/audio-analysis/5qmq...,0.715,173325,0.624,5qmq61DAAOUaW8AUo8xKhh,0.00,4,0.1230,-3.046,0,0.1140,158.087,4,https://api.spotify.com/v1/tracks/5qmq61DAAOUa...,audio_features,spotify:track:5qmq61DAAOUaW8AUo8xKhh,0.412
3,0.342,https://api.spotify.com/v1/audio-analysis/6fTt...,0.703,114893,0.594,6fTt0CH2t0mdeB2N9XFG5r,0.00,5,0.1230,-6.146,0,0.0752,153.848,4,https://api.spotify.com/v1/tracks/6fTt0CH2t0md...,audio_features,spotify:track:6fTt0CH2t0mdeB2N9XFG5r,0.475
4,0.328,https://api.spotify.com/v1/audio-analysis/2Fxm...,0.701,194088,0.425,2Fxmhks0bxGSBdJ92vM42m,0.13,7,0.1000,-10.965,1,0.3750,135.128,4,https://api.spotify.com/v1/tracks/2Fxmhks0bxGS...,audio_features,spotify:track:2Fxmhks0bxGSBdJ92vM42m,0.562


In [16]:
df_audio_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8501 entries, 0 to 8500
Data columns (total 18 columns):
acousticness        8501 non-null float64
analysis_url        8501 non-null object
danceability        8501 non-null float64
duration_ms         8501 non-null int64
energy              8501 non-null float64
id                  8501 non-null object
instrumentalness    8501 non-null float64
key                 8501 non-null int64
liveness            8501 non-null float64
loudness            8501 non-null float64
mode                8501 non-null int64
speechiness         8501 non-null float64
tempo               8501 non-null float64
time_signature      8501 non-null int64
track_href          8501 non-null object
type                8501 non-null object
uri                 8501 non-null object
valence             8501 non-null float64
dtypes: float64(9), int64(4), object(5)
memory usage: 1.2+ MB


In [17]:
columns_to_drop = ['analysis_url','track_href','type','uri']
df_audio_features.drop(columns_to_drop, axis=1,inplace=True)

df_audio_features.rename(columns={'id': 'track_id'}, inplace=True)

df_audio_features.shape

(8501, 14)

In [18]:
# merge both dataframes
# the 'inner' method will make sure that we only keep track IDs present in both datasets
df = pd.merge(df_tracks,df_audio_features,on='track_id',how='inner')
print("Shape of the dataset:", df_audio_features.shape)
df.head()

Shape of the dataset: (8501, 14)


,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Shawn Mendes,Señorita,0TK2YIli7K1leLovkQiNik,100,0.037,0.759,190960,0.540,0.00,9,0.0945,-6.039,0,0.0287,116.947,4,0.750
1,Drake,Money In The Grave (Drake ft. Rick Ross),5ry2OE6R2zPQFDO85XkgRb,92,0.101,0.831,205427,0.502,0.00,10,0.1220,-4.045,0,0.0460,100.541,4,0.101
2,Lizzo,Truth Hurts,5qmq61DAAOUaW8AUo8xKhh,90,0.110,0.715,173325,0.624,0.00,4,0.1230,-3.046,0,0.1140,158.087,4,0.412
3,Lil Nas X,Panini,6fTt0CH2t0mdeB2N9XFG5r,91,0.342,0.703,114893,0.594,0.00,5,0.1230,-6.146,0,0.0752,153.848,4,0.475
4,Billie Eilish,bad guy,2Fxmhks0bxGSBdJ92vM42m,96,0.328,0.701,194088,0.425,0.13,7,0.1000,-10.965,1,0.3750,135.128,4,0.562


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8501 entries, 0 to 8500
Data columns (total 17 columns):
artist_name         8501 non-null object
track_name          8501 non-null object
track_id            8501 non-null object
popularity          8501 non-null int64
acousticness        8501 non-null float64
danceability        8501 non-null float64
duration_ms         8501 non-null int64
energy              8501 non-null float64
instrumentalness    8501 non-null float64
key                 8501 non-null int64
liveness            8501 non-null float64
loudness            8501 non-null float64
mode                8501 non-null int64
speechiness         8501 non-null float64
tempo               8501 non-null float64
time_signature      8501 non-null int64
valence             8501 non-null float64
dtypes: float64(9), int64(5), object(3)
memory usage: 1.2+ MB


In [20]:
df[df.duplicated(subset=['artist_name','track_name'],keep=False)]

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence


In [21]:
df.to_csv('SpotifyAudioFeatures2019.csv')

In [48]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot

In [94]:
from bokeh.transform import linear_cmap
from bokeh.util.hex import hexbin

y = df['popularity']/df['popularity'].max()

def make2dplot(title,x,y):
    bins = hexbin(x, y, 0.06)
    p = figure(title=title, tools="wheel_zoom,pan,reset",
           match_aspect=True, background_fill_color='#440154')
    p.grid.visible = False
    p.hex_tile(q="q", r="r", size=0.1, line_color=None, source=bins,
           fill_color=linear_cmap('counts', 'Viridis256', 0, max(bins.counts)))
    return p

p1 = make2dplot('Danceability', df['danceability']/df['danceability'].max(), y)
p2 = make2dplot('Acousticness', df['acousticness']/df['acousticness'].max(), y)
p3 = make2dplot('Duration', df['duration_ms']/df['duration_ms'].max(), y)
p4 = make2dplot('Energy', df['energy']/df['energy'].max(), y)
p5 = make2dplot('Instrumentalness', df['instrumentalness']/df['instrumentalness'].max(), y)
p6 = make2dplot('Key', df['key']/df['key'].max(), y)
p7 = make2dplot('Valence', df['valence']/df['valence'].max(), y)
p8 = make2dplot('Liveness', df['liveness']/df['liveness'].max(), y)
p9 = make2dplot('Loudness', df['loudness']/df['loudness'].max(), y)
p10 = make2dplot('Speechiness', df['speechiness']/df['speechiness'].max(), y)
p11 = make2dplot('Tempo', df['tempo']/df['tempo'].max(), y)


output_notebook()

show(gridplot([p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,p11], ncols=4, color='red', plot_width=200, plot_height=200, toolbar_location=None))

Loading BokehJS ...

In [57]:
def make_plot(title, hist, edges, x):
    p = figure(title=title, tools='', background_fill_color="#fafafa")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],
           fill_color="navy", line_color="white", alpha=0.5)
    p.y_range.start = 0
    p.legend.location = "center"
    p.legend.background_fill_color = "#fefefe"
    p.xaxis.axis_label = 'x'
    p.yaxis.axis_label = ''
    p.grid.grid_line_color="white"
    return p

In [95]:
import numpy as np

df1=df[df['popularity']<40]

hist1, edges1 = np.histogram(df1['danceability'], bins=20)
hist2, edges2 = np.histogram(df1['valence'], bins=20)
hist3, edges3 = np.histogram(df1['acousticness'], bins=20)
hist4, edges4 = np.histogram(df1['duration_ms'], bins=20)
hist5, edges5 = np.histogram(df1['energy'], bins=20)
hist6, edges6 = np.histogram(df1['instrumentalness'], bins=20)
hist7, edges7 = np.histogram(df1['key'], bins=12)
hist9, edges9 = np.histogram(df1['liveness'], bins=20)
hist10, edges10 = np.histogram(df1['loudness'], bins=20)
hist11, edges11 = np.histogram(df1['speechiness'], bins=20)
hist12, edges12 = np.histogram(df1['tempo'], bins=20)


x = df.index

p1 = make_plot("Danceability Distribution", hist1, edges1, x)
p2 = make_plot("Valence Distribution", hist2, edges2, x)
p3 = make_plot("Acousticness Distribution", hist3, edges3, x)
p4 = make_plot("Duration Distribution", hist4, edges4, x)
p5 = make_plot("Energy Distribution", hist5, edges5, x)
p6 = make_plot("Instrumental Distribution", hist6, edges6, x)
p7 = make_plot("Key Distribution", hist7, edges7, x)
p9 = make_plot("Liveness Distribution", hist9, edges9, x)
p10 = make_plot("Loudness Distribution", hist10, edges10, x)
p11 = make_plot("Speechiness Distribution", hist11, edges11, x)
p12 = make_plot("Tempo Distribution", hist12, edges12, x)


output_notebook()

show(gridplot([p1,p2,p3,p4,p5,p6,p7,p9,p10,p11,p12], ncols=4, plot_width=200, plot_height=200, toolbar_location=None))

Loading BokehJS ...

In [96]:
df2=df[df['popularity']>=80]

hist1, edges1 = np.histogram(df2['danceability'], bins=20)
hist2, edges2 = np.histogram(df2['valence'], bins=20)
hist3, edges3 = np.histogram(df2['acousticness'], bins=20)
hist4, edges4 = np.histogram(df2['duration_ms'], bins=20)
hist5, edges5 = np.histogram(df2['energy'], bins=20)
hist6, edges6 = np.histogram(df2['instrumentalness'], bins=20)
hist7, edges7 = np.histogram(df2['key'], bins=12)
hist9, edges9 = np.histogram(df2['liveness'], bins=20)
hist10, edges10 = np.histogram(df2['loudness'], bins=20)
hist11, edges11 = np.histogram(df2['speechiness'], bins=20)
hist12, edges12 = np.histogram(df2['tempo'], bins=20)


x = df.index

p1 = make_plot("Danceability Distribution", hist1, edges1, x)
p2 = make_plot("Valence Distribution", hist2, edges2, x)
p3 = make_plot("Acousticness Distribution", hist3, edges3, x)
p4 = make_plot("Duration Distribution", hist4, edges4, x)
p5 = make_plot("Energy Distribution", hist5, edges5, x)
p6 = make_plot("Instrumental Distribution", hist6, edges6, x)
p7 = make_plot("Key Distribution", hist7, edges7, x)
p9 = make_plot("Liveness Distribution", hist9, edges9, x)
p10 = make_plot("Loudness Distribution", hist10, edges10, x)
p11 = make_plot("Speechiness Distribution", hist11, edges11, x)
p12 = make_plot("Tempo Distribution", hist12, edges12, x)


output_notebook()

show(gridplot([p1,p2,p3,p4,p5,p6,p7,p9,p10,p11,p12], ncols=4, color='red', plot_width=200, plot_height=200, toolbar_location=None))

Loading BokehJS ...

In [101]:
import json, urllib
import chart_studio.plotly as py

TypeError: iplot() missing 1 required positional argument: 'figure_or_data'